In [1]:
import pandas as pd  #load the data
from sklearn.model_selection import train_test_split   # train the data and test the data 
import time  # you can useful many time functon
import numpy as np  # this will  usefull for math func
import pickle   # because many objects cannot saved directly as text or json  it convert byte stream that can be written to file 
import matplotlib.pyplot as plt  # visaul the graph

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [3]:
# Using chi2 (a statistical measure for finding independence between categorical feature and a target variable)
def selectkbest(indep_x, dep_y, n):
    test = SelectKBest(score_func = chi2, k = n)
    fit2 = test.fit(indep_x, dep_y)
    selectk_features = fit2.transform(indep_x)
    return selectk_features

In [4]:
def split_scaler(indep_x, dep_y):
    x_train, x_test, y_train, y_test = train_test_split(indep_x, dep_y, test_size = 0.30, random_state = 0)
    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)
    return x_train, x_test, y_train, y_test 

In [5]:
def cm_prediction(classifier, x_test):
    from sklearn.linear_model import LogisticRegression
   
    y_pred = classifier.predict(x_test)
    # Making Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)
    
    from sklearn.metrics import accuracy_score
    accuracy = accuracy_score(y_test, y_pred)
    
    from sklearn.metrics import classification_report
    report = classification_report(y_test, y_pred)
    return classifier, accuracy, report, x_test, y_test, cm

In [6]:
def logistic(x_train,y_train,x_test,y_test):       
    # Fitting K-NN to the Training set
    from sklearn.linear_model import LogisticRegression
    classifier = LogisticRegression(random_state=0)
    classifier.fit(x_train, y_train)
    classifier,accuracy,report,x_test,y_test,cm = cm_prediction(classifier,x_test)
    return  classifier,accuracy,report,x_test,y_test,cm 

In [7]:
def svm_linear(x_train, y_train, x_test,y_test):
    from sklearn.svm import SVC
    classifier = SVC(kernel = 'linear', random_state = 0)
    classifier.fit(x_train, y_train)
    classifier, accuracy, report, x_test, y_test, cm = cm_prediction(classifier, x_test) 
    return  classifier,accuracy,report,x_test,y_test,cm 

In [8]:
def svm_nl(x_train, y_train, x_test,y_test):
    from sklearn.svm import SVC
    classifier = SVC(kernel = 'rbf', random_state = 0)
    classifier.fit(x_train, y_train)
    classifier, accuracy, report, x_test, y_test, cm = cm_prediction(classifier, x_test) 
    return  classifier,accuracy,report,x_test,y_test,cm 

In [9]:
def knn(x_train, y_train, x_test,y_test):
    from sklearn.neighbors import KNeighborsClassifier
    classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    classifier.fit(x_train, y_train)
    classifier, accuracy, report, x_test, y_test, cm = cm_prediction(classifier, x_test)
    return  classifier,accuracy,report,x_test,y_test,cm   


In [10]:
def naive(x_train, y_train, x_test,y_test):
    from sklearn.naive_bayes import GaussianNB
    classifier = GaussianNB()
    classifier.fit(x_train, y_train)
    classifier, accuracy, report, x_test, y_test, cm = cm_prediction(classifier, x_test)
    return  classifier,accuracy,report,x_test,y_test,cm  

In [11]:
def decision(x_train, y_train, x_test,y_test):
    from sklearn.tree import DecisionTreeClassifier
    classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
    classifier.fit(x_train, y_train)
    classifier, accuracy, report, x_test, y_test, cm = cm_prediction(classifier, x_test) 
    return  classifier,accuracy,report,x_test,y_test,cm

In [12]:
def random(x_train, y_train, x_test,y_test):
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(x_train, y_train)
    classifier, accuracy, report, x_test, y_test, cm = cm_prediction(classifier, x_test) 
    return  classifier,accuracy,report,x_test,y_test,cm  

In [13]:
def Xgboost(X_train,y_train,X_test,y_test):
    from xgboost import XGBClassifier
    from sklearn.model_selection import GridSearchCV
    param_grid = {'max_depth':[5,10,20],'learning_rate':[0.01,0.1,0.2],'subsample':[0.5,0.7,1.0],'colsample_bytree':[0.5,0.7,1.0],"gamma":[0,1]} 
    classifier = GridSearchCV(XGBClassifier(random_state=42), param_grid, refit = True,n_jobs=-1,scoring='f1_weighted') 
    classifier.fit(X_train, y_train)
    Accuracy,report,cm,f1_macro=cm_prediction(classifier,X_test,y_test)
    return classifier,Accuracy,report,cm,f1_macro

In [14]:
def AdaBoost(X_train,y_train,X_test,y_test):
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.model_selection import GridSearchCV
    param_grid={'n_estimators':[50,100,200],"learning_rate":[0.01,0.1,0.5]}
    classifier = GridSearchCV(AdaBoostClassifier(random_state=42),param_grid,refit=True,n_jobs=-1,scoring='f1_weighted')
    classifier.fit(X_train,y_train)
    Accuracy,report,cm,f1_macro=cm_prediction(classifier,X_test,y_test)
    return classifier,Accuracy,report,cm,f1_macro   

In [15]:
def selectk_classification(acclog, accsvml, accsvmnl, accknn, accnaive, accdeci, accrand):
    dataframe = pd.DataFrame(index = ['ChiSquare'], columns = ['Logistic', 'SVML', 'SVMNL', 'KNN', 'Naive', 'Decision', 'Random'])
    for number, item in enumerate(dataframe.index):       # Enumerate - it is a counter that adds index to each items in an iterable
        dataframe['Logistic'][item] = acclog[number]
        dataframe['SVML'][item] = accsvml[number]
        dataframe['SVMNL'][item] = accsvmnl[number]
        dataframe['KNN'][item] = accknn[number]
        dataframe['Naive'][item] = accnaive[number]
        dataframe['Decision'][item] = accdeci[number]
        dataframe['Random'][item] = accrand[number]
        dataframe.loc[idex,'XgBoost']=accxg[number]
        dataframe.loc[idex,'AdaBoost']=accada[number]
    return dataframe        

In [16]:
dataset = pd.read_csv('Water_cleaned_final.csv', index_col = None) 
dataset

,Household_ID,members,Daily_Water_Consumption_Liters,Temperature_C,Humidity_percent,water source type,drinking_water_ use _Liters,Appliance_Usage_Water_Liters,Day_type
0,1,2,420,26.4,55,Municipal,12,65,weekday
1,2,3,510,27.1,60,Borewell,15,70,Weekday
2,3,4,630,28.9,58,Municipal,18,75,Weekend
3,4,5,720,30.2,62,Tanker,20,80,Weekday
4,5,3,460,25.8,50,Borewell,14,60,Weekday
...,...,...,...,...,...,...,...,...,...
95,96,6,860,34.8,74,Borewell,24,75,Weekday
96,97,2,380,24.8,50,Tanker,11,65,Weekend
97,98,4,560,28.5,60,Municipal,16,80,Weekday
98,99,3,490,26.4,53,Borewell,13,70,Weekend


In [17]:
df1 = dataset
df1 = pd.get_dummies(df1, dtype = int, drop_first = True)

In [18]:
df1



,Household_ID,members,Daily_Water_Consumption_Liters,Temperature_C,Humidity_percent,drinking_water_ use _Liters,Appliance_Usage_Water_Liters,water source type_Municipal,water source type_Tanker,Day_type_Weekend,Day_type_weekday
0,1,2,420,26.4,55,12,65,1,0,0,1
1,2,3,510,27.1,60,15,70,0,0,0,0
2,3,4,630,28.9,58,18,75,1,0,1,0
3,4,5,720,30.2,62,20,80,0,1,0,0
4,5,3,460,25.8,50,14,60,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
95,96,6,860,34.8,74,24,75,0,0,0,0
96,97,2,380,24.8,50,11,65,0,1,1,0
97,98,4,560,28.5,60,16,80,1,0,0,0
98,99,3,490,26.4,53,13,70,0,0,1,0


In [19]:
df1.columns





Index(['Household_ID', 'members ', 'Daily_Water_Consumption_Liters',
       'Temperature_C', 'Humidity_percent', 'drinking_water_ use _Liters ',
       'Appliance_Usage_Water_Liters', 'water source  type_Municipal',
       'water source  type_Tanker', 'Day_type_Weekend', 'Day_type_weekday'],
      dtype='object')

In [20]:
indep_x = df1.drop("Day_type_Weekend", axis=1)
dep_y = df1["Day_type_Weekend"]

In [21]:
kbest = SelectKBest(score_func=f_classif, k=7)
new_indep = kbest.fit_transform(indep_x, dep_y)

new_indep = kbest.transform(indep_x)






In [22]:
x_train, x_test, y_train, y_test = split_scaler(new_indep, dep_y)

In [23]:

acclog = []
accsvml = []
accsvmnl = []
accknn = []
accnaive = []
accdeci = []
accrand = []
accxg  = []
accada  =[]


In [24]:
classifier,Accuracy,report,cm,f1_macro=svml(X_train,y_train,X_test,y_test)  
accsvml.append(Accuracy)

classifier,Accuracy,report,cm,f1_macro=svmnl(X_train,y_train,X_test,y_test)  
accsvmnl.append(Accuracy)
    
classifier,Accuracy,report,cm,f1_macro=knn(X_train,y_train,X_test,y_test)  
accknn.append(Accuracy)
    
classifier,Accuracy,report,cm,f1_macro=Naive(X_train,y_train,X_test,y_test)  
accnav.append(Accuracy)
    
classifier,Accuracy,report,cm,f1_macro=Decision(X_train,y_train,X_test,y_test)  
accdes.append(Accuracy)
    
classifier,Accuracy,report,cm,f1_macro=random(X_train,y_train,X_test,y_test)  
accrf.append(Accuracy)

classifier,Accuracy,report,cm,f1_macro=Xgboost(X_train,y_train,X_test,y_test)  
accxg.append(Accuracy)

classifier,Accuracy,report,cm,f1_macro=AdaBoost(X_train,y_train,X_test,y_test)  
accada.append(Accuracy)

NameError: name 'svml' is not defined

In [ ]:
result = selectk_classification(acclog, accsvml, accsvmnl, accknn, accnaive, accdeci, accrand)

In [ ]:
result